THƯ VIỆN

In [17]:
import os
import glob
import time
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.fftpack as fftpack
import scipy.signal as signal
import librosa
import librosa.display
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold, cross_val_score
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import ExtraTreeClassifier
import xgboost as xgb

STFT

In [18]:
def create_spectrogram_features(file_path, sr=16000, n_fft=512, hop_length=160, win_length=400, max_freq=1200, return_spectrogram=False):
    try:
        # Đọc tệp âm thanh
         y, sr = librosa.load(file_path, sr=sr)
        
         # Tính STFT
         D = librosa.stft(y, n_fft=n_fft, hop_length=hop_length, win_length=win_length, window='hann')
       
         # Chuyển sang biên độ
         S = np.abs(D)
        
         # Giới hạn tần số từ 0 đến 1200 Hz
         freqs = librosa.fft_frequencies(sr=sr, n_fft=n_fft)
         max_bin = np.where(freqs <= max_freq)[0][-1]  # Tìm bin tối đa <= 1200 Hz
         S_limited = S[:max_bin + 1, :]  # Cắt ma trận
       
         # Chuyển sang thang dB
         S_db = librosa.amplitude_to_db(S_limited, ref=np.max)
        
         if return_spectrogram:
             # Trả về spectrogram đầy đủ cho visualization
             return S_db, freqs[:max_bin + 1], sr
         else:
             # Trích xuất đặc trưng: trung bình theo thời gian cho mỗi bin tần số
             features = np.mean(S_db, axis=1)
             return features
    except Exception as e:
        print(f"Lỗi khi xử lý {file_path}: {e}")
        return None

ML MODEL

In [19]:
logistic = LogisticRegression(max_iter = 1500)
svm = SVC(kernel = 'rbf', C = 1, gamma = 'scale')
knn = KNeighborsClassifier(n_neighbors = 5)
random_forest = RandomForestClassifier()
extra_tree = ExtraTreeClassifier()
xgboost = xgb.XGBClassifier()
models = {
    "Logistic Regression": logistic,
    "SVM": svm,
    "KNN": knn,
    "Random Forest": random_forest,
    "Extra Tree": extra_tree,
    "XGBoost": xgboost,
}

ĐỌC DỮ LIỆU VÀ TRÍCH XUẤT ĐẶC TRƯNG

In [ ]:
# Đường dẫn dữ liệu và các nhãn
base_path = r"nuhive_processed/"
categories = [r"bee", r"nobee", r"noqueen"]

# Trích xuất đặc trưng từ tất cả các tệp âm thanh
X = []
labels = []
file_paths = []
print("Đang trích xuất đặc trưng từ các tệp âm thanh...")
for category in categories:
    files = glob.glob(os.path.join(base_path, category, "*.wav"))
    for file in tqdm(files, desc=f"Xử lý {category}"):
        features = create_spectrogram_features(file)
        if features is not None:
            X.append(features)
            labels.append(category)
            file_paths.append(file)

# Chuyển đổi dữ liệu thành numpy array, sử dụng label encoder cho nhãn
X = np.array(X)
y_encoder = LabelEncoder()
y = y_encoder.fit_transform(labels)

print(f"Tổng số mẫu: {len(X)}")
print(f"Kích thước đặc trưng: {X.shape[1]}")


Đang trích xuất đặc trưng từ các tệp âm thanh...


Xử lý bee:   0%|          | 0/5473 [00:00<?, ?it/s]

Xử lý noqueen: 100%|██████████| 4861/4861 [00:18<00:00, 256.67it/s]

Tổng số mẫu: 13792
Kích thước đặc trưng: 39


HUẤN LUYỆN VÀ BÁO CÁO KẾT QUẢ

In [26]:
models_name = ["Logistic Regression", "SVM", "KNN", "Random Forest", "Extra Tree", "XGBoost"]
f1_score = []
accuracy = []
training_time = []

# Đánh giá bằng KFold
k_fold = KFold(n_splits=5, shuffle=True, random_state=42)

# Huấn luyện và đánh giá từng mô hình
for model_name, model in models.items():
    print(f"Đang huấn luyện và đánh giá mô hình {model_name}:")
    start_time = time.time()
    cv_f1_scores = cross_val_score(model, X, y, cv=k_fold, scoring='f1_macro')
    cv_accuracy_scores = cross_val_score(model, X, y, cv=k_fold, scoring='accuracy')
    end_time = time.time()
    timing = end_time - start_time
    f1_score.append(np.mean(cv_f1_scores))
    accuracy.append(np.mean(cv_accuracy_scores))
    training_time.append(timing)
    print(f"Xong")

# Tạo DataFrame để hiển thị kết quả
results_df = pd.DataFrame({
    "Model": models_name,
    "F1 Score": f1_score,
    "Accuracy": accuracy,
    "Training Time (s)": training_time
})

print("\nKẾT QUẢ ĐÁNH GIÁ MÔ HÌNH")
results_df.style

Đang huấn luyện và đánh giá mô hình Logistic Regression:
Xong
Đang huấn luyện và đánh giá mô hình SVM:
Xong
Đang huấn luyện và đánh giá mô hình KNN:
Xong
Đang huấn luyện và đánh giá mô hình Random Forest:
Xong
Đang huấn luyện và đánh giá mô hình Extra Tree:
Xong
Đang huấn luyện và đánh giá mô hình XGBoost:
Xong

KẾT QUẢ ĐÁNH GIÁ MÔ HÌNH


,Model,F1 Score,Accuracy,Training Time (s)
0,Logistic Regression,0.685360,0.726508,33.533210
1,SVM,0.734578,0.777335,35.499151
2,KNN,0.784367,0.815328,0.585484
3,Random Forest,0.785423,0.819388,89.529107
4,Extra Tree,0.666581,0.691488,0.178494
5,XGBoost,0.790092,0.818663,8.820579
